In [1]:
import geopandas as gpd
import glob
import folium
import ipywidgets as widgets
import json
import matplotlib.pyplot as plt
import os
import xarray as xr

In [2]:
import benchmarks
import calculate_metrics
import visualization
import uruguay as provider

In [3]:
import psutil

# Get the available memory in bytes
available_memory_bytes = psutil.virtual_memory().available

# Convert to gigabytes for readability
available_memory_gb = available_memory_bytes / 1024 ** 3

print(f"Available memory: {available_memory_gb:.2f} GB")

Available memory: 6.18 GB


# Load Experiment Data

## Load Basin & Gaguge Shapefile

In [4]:
with open(provider.SHAPEFILE_PATH, 'rb') as f:
  geography_gdf = gpd.read_file(f)
geography_gdf.set_index('unique_gauge_id', inplace=True)
geography_gdf

,area,gauge_longitude,gauge_latitude,drain_area,station_code,provider,data_source,altitude,river,gauge_serial_number,...,DIST_SINK,DIST_MAIN,gauge_sub_basin_area,hybas_upstream_area,gauge_pfaf_id,ENDO,COAST,ORDER,SORT,geometry
unique_gauge_id,,,,,,,,,,,,,,,,,,,,,
DNAUY_52,6687.0,-55.470990,-31.875730,None,52,DNAUY,uruguay_metadata_jan_2024.csv,None,None,None,...,559.5,559.5,111.9,6687.5,641608700300,0,0,2,87210,"POLYGON ((-55.53333 -31.9875, -55.53971 -31.98..."
DNAUY_1743,11630.0,-55.226580,-32.443560,None,1743,DNAUY,uruguay_metadata_jan_2024.csv,None,None,None,...,525.4,525.4,155.1,11629.7,641609303000,0,0,1,87172,"POLYGON ((-54.79583 -32.84583, -54.8 -32.84583..."
DNAUY_46.1,872.7,-54.900800,-34.769400,None,46.1,DNAUY,uruguay_metadata_jan_2024.csv,None,None,None,...,12.8,12.8,79.0,872.8,641503403000,0,0,1,86601,"POLYGON ((-54.92083 -34.825, -54.925 -34.825, ..."
DNAUY_174,487.1,-54.953160,-34.747600,None,174,DNAUY,uruguay_metadata_jan_2024.csv,None,None,None,...,12.8,12.8,140.8,487.3,641503402100,0,0,2,86602,"POLYGON ((-54.925 -34.82083, -54.92917 -34.820..."
DNAUY_44,2812.8,-55.741100,-34.216100,None,44,DNAUY,uruguay_metadata_jan_2024.csv,None,None,None,...,131.1,131.1,214.9,2812.5,641504709000,0,0,1,86717,"POLYGON ((-54.99167 -34.42083, -54.99167 -34.4..."
DNAUY_59.1,4914.1,-56.249200,-34.364700,None,59.1,DNAUY,uruguay_metadata_jan_2024.csv,None,None,None,...,75.5,75.5,83.0,4913.8,641504701300,0,0,1,86673,"POLYGON ((-54.99167 -34.42083, -54.99167 -34.4..."
DNAUY_133,8389.9,-56.401100,-34.449700,None,133,DNAUY,uruguay_metadata_jan_2024.csv,None,None,None,...,43.6,43.6,42.7,8390.0,641504500100,0,0,1,86658,"POLYGON ((-54.99167 -34.42083, -54.99167 -34.4..."
DNAUY_53.1,1825.1,-56.203300,-34.091100,None,53.1,DNAUY,uruguay_metadata_jan_2024.csv,None,None,None,...,95.7,95.7,233.2,1825.3,641504603000,0,0,2,86683,"POLYGON ((-56.19167 -34.16667, -56.19223 -34.1..."
DNAUY_28.0,219910.3,-57.610707,-30.260656,None,28.0,DNAUY,uruguay_metadata_jan_2024.csv,None,None,None,...,340.9,340.9,209.7,219907.4,641819090000,0,0,1,87662,"MULTIPOLYGON (((-57.07379 -28.62443, -57.07042..."


## Load Performance Metrics

In [5]:
metrics_file = f'{provider.EXPERIMENT_RESULTS_PATH}/metrics.nc'
with open(metrics_file, 'rb') as f:
  metrics_ds = xr.open_dataset(f)

## Dimensions

In [6]:
experiments = metrics_ds.experiment.values.tolist()
gauges = metrics_ds.gauge_id.values.tolist()
lead_times = metrics_ds.lead_time.values.tolist()
available_metrics = list(metrics_ds.data_vars)

## Load Hydrographs

In [7]:
hydrographs = {
    experiment: calculate_metrics.load_model_runs_for_experiment(
        experiment=experiment,
        gauges=provider.TRAIN_GAUGES,
        model_run_base_path=provider.MODEL_RUN_DIRECTORY,
    ).assign_coords(experiment=experiment)
    for experiment in experiments if experiment not in ['persistence', 'monthly_climatology']
}
hydrographs = xr.concat(hydrographs.values(), dim='experiment')
hydrographs

<xarray.Dataset>
Dimensions:      (experiment: 4, gauge_id: 19, time: 2922, lead_time: 8)
Coordinates:
  * time         (time) datetime64[ns] 2016-01-01 2016-01-02 ... 2023-12-31
  * lead_time    (lead_time) int32 0 1 2 3 4 5 6 7
  * gauge_id     (gauge_id) object 'DNAUY_52' 'DNAUY_1743' ... 'DNAUY_176.1'
  * experiment   (experiment) <U41 'gauged' ... 'hydrologically_separated_cro...
Data variables:
    observation  (experiment, gauge_id, time, lead_time) float64 8.427 ... nan
    prediction   (experiment, gauge_id, time, lead_time) float64 9.4 ... 0.04912

# Basin Geography



In [8]:
@widgets.interact(
    gauge_options = widgets.SelectMultiple(
        options=gauges,
        value=gauges,
        description='Gauge ID',
    )
)
def interactive_basin_and_gauge_geography(
    gauge_options,
):
  gauges = list(gauge_options)
  if len(gauge_options) == 1:
    map_center = (
          geography_gdf.loc[gauges[0]].gauge_latitude,
          geography_gdf.loc[gauges[0]].gauge_longitude
      )
    initial_map_zoom = provider.MAP_ZOOM + 3
  else:
    map_center = provider.MAP_CENTER
    initial_map_zoom = provider.MAP_ZOOM

  basin_map = visualization.basin_and_gauge_geography(
      geography=geography_gdf,
      gauges=gauge_options,
      map_center=map_center,
      initial_map_zoom=initial_map_zoom,
      height=500,
      width='50%'
  )
  return basin_map.add_child(folium.LayerControl())

interactive(children=(SelectMultiple(description='Gauge ID', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, …

# Gauge Hydrographs
This is the section that does not work because of the NetCDF compatability issue mentioned above.

In [9]:
# Select gauge to view hydrographs.
gauge = gauges[0]
gauge_selector = widgets.widgets.Dropdown(
    options=gauges, description='Gauge ID')
def on_selector_change(change):
  global gauge
  gauge = change['new']
gauge_selector.observe(on_selector_change, names='value')
display(gauge_selector)

Dropdown(description='Gauge ID', options=('DNAUY_52', 'DNAUY_1743', 'DNAUY_46.1', 'DNAUY_174', 'DNAUY_44', 'DN…

In [10]:
# Re-run this cell after making a selection from the dropdown box above.
visualization.plot_hydrographs_for_gauge(
    hydrographs=hydrographs,
    gauge=gauge,
    experiments=experiments,
)

# Forecast Timeseries

In [11]:
# Select gauge to view hydrographs.
gauge = gauges[0]
gauge_selector = widgets.widgets.Dropdown(
    options=gauges, description='Gauge ID')
def on_selector_change(change):
  global gauge
  gauge = change['new']
gauge_selector.observe(on_selector_change, names='value')
display(gauge_selector)

Dropdown(description='Gauge ID', options=('DNAUY_52', 'DNAUY_1743', 'DNAUY_46.1', 'DNAUY_174', 'DNAUY_44', 'DN…

In [12]:
gauge = 'DNAUY_52'
experiment = 'gauged'
start_time_idx = 1000
time_window_days = 200

@widgets.interact(
    gauge_options = widgets.Select(
        options=gauges,
        value=gauges[0],
        description='Gauge ID',
    ),
    experiment_options=widgets.Select(
        options=experiments,
        value=experiments[0],
        description='Experiment',    
    ),
    start_time_options=widgets.IntSlider(
        min=0,
        max=len(hydrographs.time),
        step=1,
        value=0
    ),
    time_window_options=widgets.IntSlider(
        min=0,
        max=180,
        step=1,
        value=90
    )
)
def interactive_plot_cross_validation_splits(
    gauge_options,
    experiment_options,
    start_time_options,
    time_window_options,
):
    visualization.plot_forecast_traces(
        hydrographs=hydrographs,
        experiment=experiment_options,
        gauge=gauge_options,
        start_time_idx=start_time_options,
        time_window_days=time_window_options
    )

interactive(children=(Select(description='Gauge ID', options=('DNAUY_52', 'DNAUY_1743', 'DNAUY_46.1', 'DNAUY_1…

# Gauge Metrics

In [13]:
@widgets.interact(
    metrics = widgets.SelectMultiple(
        options=available_metrics,
        value=visualization.DEFAULT_METRICS,
    )
)
def interactive_scores_table(
    metrics,
    gauge=gauges,
    lead_time=lead_times,
):
  visualization.scores_table(
      gauge=gauge,
      lead_time=lead_time,
      metrics=metrics,
      metrics_ds=metrics_ds,
  )

interactive(children=(SelectMultiple(description='metrics', index=(2, 6, 4, 5, 20, 32), options=('MSE', 'RMSE'…

# Inspect Experiments

## Map of Cross Validation Splits

In [14]:
# Load mapping from gauge to model run directory.
with open(provider.GAUGE_TO_MODEL_PATH_MAPPING_FILE, 'r') as f:
  gauge_to_model_path = json.load(f)

In [15]:
@widgets.interact
def interactive_plot_cross_validation_splits(
    experiment=gauge_to_model_path.keys(),
):
  gauge_to_split = {
      gauge: os.path.basename(model_path)
        for gauge, model_path in gauge_to_model_path[experiment].items()
  }
  cross_validation_splits = {split: [] for split in set(gauge_to_split.values())}
  for gauge, split in gauge_to_split.items():
    cross_validation_splits[split].append(gauge)

  basin_map = visualization.plot_cross_validation_experiment_folium(
    splits=cross_validation_splits,
    geography=geography_gdf,
    map_center=provider.MAP_CENTER,
    initial_map_zoom=provider.MAP_ZOOM,
    height=500,
    width='50%',
  )
  return basin_map.add_child(folium.LayerControl())

interactive(children=(Dropdown(description='experiment', options=('gauged', 'ungauged', 'random_cross_validati…

In [16]:
@widgets.interact
def interactive_plot_cross_validation_splits(
    experiment=gauge_to_model_path.keys(),
):
  gauge_to_split = {gauge: os.path.basename(model_path) for gauge, model_path in gauge_to_model_path[experiment].items()}
  cross_validation_splits = {split: [] for split in set(gauge_to_split.values())}
  for gauge, split in gauge_to_split.items():
    cross_validation_splits[split].append(gauge)

  visualization.plot_cross_validation_experiment_cartoon(
    splits=cross_validation_splits,
    geography=geography_gdf,
    provider_countries=provider.COUNTRIES,
  )

interactive(children=(Dropdown(description='experiment', options=('gauged', 'ungauged', 'random_cross_validati…

## Score Map

In [17]:
@widgets.interact
def interactive_score_map(
    metric=available_metrics,
    experiment=experiments,
    lead_time=lead_times,
):
  visualization.score_map(
      basin_geometries=geography_gdf,
      experiment=experiment,
      metric=metric,
      lead_time=lead_time,
      metrics=metrics_ds,
      provider_countries=provider.COUNTRIES,
  )

interactive(children=(Dropdown(description='metric', options=('MSE', 'RMSE', 'NSE', 'log-NSE', 'Beta-NSE', 'Al…

## Score Distribution Boxplots

In [18]:
@widgets.interact(
    experiments = widgets.SelectMultiple(
        options=experiments,
        value=experiments,
        desctiption='Experiment(s)'
    )
)
def interactive_score_distribution_box_plot(
    experiments,
    metric=available_metrics,
):
  visualization.score_distribution_box_plot(
      metrics=metrics_ds,
      experiments=experiments,
      gauges=gauges,
      metric=metric,
  )

interactive(children=(SelectMultiple(description='experiments', index=(0, 1, 2, 3, 4, 5), options=('gauged', '…



## Score Distribution CDFs

In [19]:
@widgets.interact(
    experiments_options = widgets.SelectMultiple(
        options=experiments,
        value=experiments,
        desctiption='Experiment(s)'
    ),
    lead_times_options = widgets.SelectMultiple(
        options=lead_times,
        value=lead_times[:1],
        desctiption='Lead Time(s)'
    )
)
def interactive_plot_cdfs(
    experiments_options,
    lead_times_options,
):
  visualization.plot_cdfs(
      metrics=metrics_ds[visualization.DEFAULT_METRICS],
      experiments=experiments_options,
      lead_times=lead_times_options,
      gauges=gauges,
  )
  plt.show()

interactive(children=(SelectMultiple(description='experiments_options', index=(0, 1, 2, 3, 4, 5), options=('ga…



## Score Difference Map

In [20]:
@widgets.interact
def interactive_score_map(
    metric=available_metrics,
    experiment=experiments,
    baseline_experiment=experiments,
    lead_time=lead_times,
):
  visualization.score_map(
      basin_geometries=geography_gdf,
      experiment=experiment,
      baseline_experiment=baseline_experiment,
      metric=metric,
      lead_time=lead_time,
      metrics=metrics_ds,
      provider_countries=provider.COUNTRIES,
  )

interactive(children=(Dropdown(description='metric', options=('MSE', 'RMSE', 'NSE', 'log-NSE', 'Beta-NSE', 'Al…